<a href="https://colab.research.google.com/github/AmrRagab0/AI-Zewail-City-Map-simulation/blob/master/AI_Project_final_Version_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

### The nodes (building - road)

In [ ]:
class block():
  child = None
  parent = None
  cost = None
  action = None

class building(block):
  Type = 'building'
  def __init__(self,name,rooms,h):
    self.name = name
    self.rooms =rooms
    self.h =h


class road(block):
  Type = 'road'
  def __init__(self,length,name = 'road' ,h=0):
    self.length = length
    self.name = name
    self.h = h  # =0 by default ,no need to assign zero



### Complete ZC Map

In [ ]:


# all buidings
Dorms = building('Dorms',['room 207','room 401'],0)
Service = building('service',['room f13','room g16','room g17','room g18','ATM machine'],0)
Nano = building('Nano',['room s13','room s28','room s32','room f25','Wireless lab','Cleen room'],0)
Helmy = building('Helmy',['bio lab','chemistry lab','room s08'],0)
Complex = building('Complex',["Dean's office, Graduation Hall,IT office"],0)
GateA = building('Gate A',['security office','Rest room'],0)
Academic = building('Academic',['room G19','room G20','room G21','Biology lab'],0)
OneStopShop = building('OneStopShop',['room G10','room G11','room G12','Computer lab','Admission office'],0)


# suggested format for roads : r_2letters of one building_2 letters of other building
r_GA_CO = road(10,'GA_CO')
r_AC_CO = road(3,'AC_CO')
r_AC_DO = road(20,'AC_DO')

r_DO_SE= road(17,'DO_SE')
r_DO_NA= road(9,'DO_NA')
r_NA_ON= road(13,"NA_ON")
road2= road(2,"road2")
road3= road(6,"road3")
road6= road(7,"road6")
road7= road(43,"road7")
road8= road(3,"road8")
road9= road(9,"road9")



In [ ]:
r_DO_SE.Type

'road'

In [ ]:
class ZC_problem:
  # Complete Map
  ZC = [ [Dorms  ,                   r_DO_SE  ,             Service],
       [r_DO_NA,                     road2 ,                road3  ],
       [Nano,                        Academic  ,     	      Helmy],
       [r_NA_ON,                     Complex ,              road7],
       [OneStopShop,                 r_GA_CO ,              road6],
       [road9  ,   			             GateA,                 road8]]

  def __init__(self,init_state,goal_state):

    self.init_state = self.find(init_state)
    self.goal_state = self.find(goal_state)




  def actions(self,state):
    # get possible actions

    r,c = state[0],state[1]
    possible_moves = []
    try:
      if self.ZC[r+1][c]:
        possible_moves.append('down')
    except :
      pass
    try:
      if r-1 >=0:
        possible_moves.append('up')
    except :
      pass
    try:
      if self.ZC[r][c+1]:
        possible_moves.append('right')
    except :
      pass
    try:
      if c-1 >= 0:
        possible_moves.append('left')
    except :
      pass
    return possible_moves

  def result(self,state,action):
    # returns next state based on action
    r,c = state[0],state[1]
    if action == 'up':
      new_state = (r-1,c)
    if action == 'down':
      new_state = (r+1,c)
    if action == 'right':
      new_state = (r,c+1)
    if action == 'left':
      new_state = (r,c-1)
    return new_state


  def get_child(self,parent,action):
    child = self.result(parent,action)
    r,c = child
    parent_node= self.ZC[parent[0]][parent[1]]
    (self.ZC[r][c]).parent = parent_node
    #print(type(parent))
    parent_node.child = self.ZC[r][c]
    print(f'parent : {parent_node}')
    parent_node.action = action
    print(f'action : {action}')
    return r,c



  def found_room(self,state,room):   # tested and working
    # checks the current state and sees if the room needed is in that state, if found return true
    room = room.lower() #lower case to avoid mismatches
    state_rooms = map(lambda x : x.lower(), state.rooms)
    if room in state_rooms :
      return True
    else :
      return False

  def find(self,name):
    # finds the node index
    # given a name of a building or a road, it returns its index in the map
    name = name.lower()
    for i in range(len(self.ZC)):
      for j in range(len(self.ZC[0])):
        if (self.ZC[i][j]).Type == "road":
          continue
        if name == (self.ZC[i][j].name).lower():
          return i,j
        if name in (self.ZC[i][j]).rooms:
          return i,j



  def generate_h(self, init_state, goal_state):
    # according to goal and current state, generate heuristic for each cell
    x2,y2 = goal_state[0],goal_state[1]
    for i in range(len(self.ZC)):
      for j in range(len(self.ZC[i])):
        x1,y1 = i,j
        new_h=round(math.sqrt(((x1-x2)**2)+((y1-y2)**2)),2) # Manhattan distance equation
        #print(f'new h : {new_h}')
        self.ZC[i][j].h = new_h

  def goal_test(self, state):
    # returns true if we are in the right building
    # or if we are in the right room inside a building
    return (state == self.goal_state) #or (self.found_room(state))


  def calculate_cost(self,init):
    # not sure if we need that function
    var=0


  def heuristic(self,state):
    # just returns the heuristic parameter for each node
    return state.h


  def solution(self,node):
    '''A method to extract the sequence of actions representing the solution from the goal node.'''
    actions = []
    #cost = node.path_cost
    while node.parent is not None:
        print('current node',node)
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    return actions

  def cumulativeCost(self, state):
    CCost = 0
    ''' it checks if the node is a road type then it add its length to the total cost the algorith will move to the next state unitl it reaches the goal state
    it takes a each state in the path and the algorithm call the function at each node it reaches
    '''
    if state.Type == 'road':
      CCost = CCost + state.length
    return CCost

### Visualizer

In [ ]:
from shutil import get_terminal_size
terminal_width, _ = get_terminal_size()

_visualizers = {}

def _default_visualizer(_, state):
    '''Generic visualizer for unknown problems.'''
    print(state)

class Visualizer:
    '''Visualization and printing functionality encapsulation.'''

    def __init__(self, problem):
        '''Constructor with the problem to visualize.'''
        self.problem = problem
        self.counter = 0

    def visualize(self, frontier):
        '''Visualizes the frontier at every step.'''
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for state in frontier:
            print()
            _visualizers.get(type(self.problem), _default_visualizer)(self.problem, state)
        print('-' * terminal_width)

In [ ]:
from collections import deque

## BFS

In [ ]:
from collections import deque
stations = []
stations_indecies = []
def bfs_graph(problem, verbose=False):
    '''Breadth-first graph search implementation.'''
    #explored = []

    #print("problem.init_state",problem.init_state)





    if problem.goal_test(problem.init_state):
      return problem.solution(problem.init_state)

    frontier = deque()
    frontier.append(tuple(problem.init_state))


    #print(problem.init_state)
    #explored.append(problem.init_state)
    explored = {problem.init_state}
    #added:


    #
    #frontier = deque(tuple([(problem.init_state)]))
    #print("frontier",frontier)
    #explored = {problem.init_state}
    if verbose: visualizer = Visualizer(problem)

    while frontier:

      if verbose: visualizer.visualize(frontier)
      node = frontier.pop()
      print("now we are at",problem.ZC[node[0]][node[1]].name)
      stations.append(problem.ZC[node[0]][node[1]].name)
      stations_indecies.append(tuple(node))
      #print('I am here:',node)
      for action in problem.actions(node):
          child = problem.result(node, action)
          problem.ZC[node[0]][node[1]].child = problem.ZC[child[0]][child[1]]
          problem.ZC[child[0]][child[1]].parent = problem.ZC[node[0]][node[1]]
          print('child is', child)
          if child not in explored:
            if problem.goal_test(child):

              print("now we are at",problem.ZC[child[0]][child[1]].name)
              stations.append(problem.ZC[child[0]][child[1]].name)
              stations_indecies.append(tuple(node))
              print()
              return
              return problem.solution(child)
            frontier.appendleft(child)
            explored.add(child)

In [ ]:
init_state = 'Dorms'
goal_state = "Gate A"
test = ZC_problem(init_state, goal_state)

In [ ]:
bfs_graph(test, verbose=True)

Frontier at step 1

(0, 0)
--------------------------------------------------------------------------------
now we are at Dorms
child is (1, 0)
child is (0, 1)
Frontier at step 2

(0, 1)

(1, 0)
--------------------------------------------------------------------------------
now we are at DO_NA
child is (2, 0)
child is (0, 0)
child is (1, 1)
Frontier at step 3

(1, 1)

(2, 0)

(0, 1)
--------------------------------------------------------------------------------
now we are at DO_SE
child is (1, 1)
child is (0, 2)
child is (0, 0)
Frontier at step 4

(0, 2)

(1, 1)

(2, 0)
--------------------------------------------------------------------------------
now we are at Nano
child is (3, 0)
child is (1, 0)
child is (2, 1)
Frontier at step 5

(2, 1)

(3, 0)

(0, 2)

(1, 1)
--------------------------------------------------------------------------------
now we are at road2
child is (2, 1)
child is (0, 1)
child is (1, 2)
child is (1, 0)
Frontier at step 6

(1, 2)

(2, 1)

(3, 0)

(0, 2)
------

In [ ]:
for i in range(len(stations)):
  print('the current state is:', stations[i])

the current state is: Dorms
the current state is: DO_NA
the current state is: DO_SE
the current state is: Nano
the current state is: road2
the current state is: service
the current state is: NA_ON
the current state is: Academic
the current state is: road3
the current state is: OneStopShop
the current state is: Complex
the current state is: Helmy
the current state is: road9
the current state is: Gate A


In [ ]:
stations_indecies

[(0, 0),
 (1, 0),
 (0, 1),
 (2, 0),
 (1, 1),
 (0, 2),
 (3, 0),
 (2, 1),
 (1, 2),
 (4, 0),
 (3, 1),
 (2, 2),
 (5, 0),
 (5, 0)]

In [ ]:
def path_actions(path):
  actions_steps = []
  for i in range(len(path)):
    if i == len(path)-1:
      break
    else:
      #if path[i][0] == path[i+1][0]:
        if path[i][1] < path[i+1][1]:
          actions_steps.append('right')
        else:
          actions_steps.append('left')
      #if path[i][1] == path[i+1][1]:
        if path[i][0] < path[i+1][0]:
          actions_steps.append('down')
        else:
          actions_steps.append('up')
  return actions_steps

In [ ]:
path_actions(stations_indecies)

['left',
 'down',
 'right',
 'up',
 'left',
 'down',
 'right',
 'up',
 'right',
 'up',
 'left',
 'down',
 'right',
 'up',
 'right',
 'up',
 'left',
 'down',
 'right',
 'up',
 'right',
 'up',
 'left',
 'down',
 'left',
 'up']

## DFS

In [ ]:
def DFS(problem, verbose=False):
    '''Depth-first graph search implementation.'''
    if problem.goal_test(problem.init_state): return problem.solution(problem.init_state)
    frontier = deque()
    frontier.append(tuple(problem.init_state))
    explored = {problem.init_state}
    #if verbose: visualizer = Visualizer(problem)
    while frontier:
      #if verbose: visualizer.visualize(frontier)
      node = frontier.popleft()
      print("now we are at",problem.ZC[node[0]][node[1]].name)
      for action in problem.actions(node):
        child = problem.result(node, action)
        problem.ZC[node[0]][node[1]].child = problem.ZC[child[0]][child[1]]
        problem.ZC[child[0]][child[1]].parent = problem.ZC[node[0]][node[1]]
        #print(explored)
        if child not in explored:
          if problem.goal_test(child):
              print('Now we reached ',problem.ZC[child[0]][child[1]].name)
              return
              #return problem.solution(problem.ZC[child[0]][child[1]])
          frontier.appendleft(child)
          #print('child before adding',child)
          explored.add(child)

In [ ]:
init_state = 'Dorms'
goal_state = "Gate A"

first_try = ZC_problem(init_state,goal_state)
%prun DFS(first_try)

now we are at Dorms
now we are at DO_SE
now we are at service
now we are at road3
now we are at Helmy
now we are at Academic
now we are at Nano
now we are at NA_ON
now we are at OneStopShop
now we are at GA_CO
Now we reached  Gate A
 

## IDS

In [ ]:
def DLS(problem, limit):
    '''Depth-limited graph search imCCostplementation.'''
    if problem.goal_test(problem.init_state): return problem.solution(problem.init_state)
    frontier = deque()
    frontier.append(tuple(problem.init_state))
    explored = {problem.init_state}
    while frontier:
      node = frontier.popleft()
      print("now we are at",problem.ZC[node[0]][node[1]].name)
      for action in problem.actions(node):
        child = problem.result(node, action)
        problem.ZC[node[0]][node[1]].child = problem.ZC[child[0]][child[1]]
        problem.ZC[child[0]][child[1]].parent = problem.ZC[node[0]][node[1]]
        #print(explored)
        if child not in explored:
          if problem.goal_test(child):
              print('Now we reached ',problem.ZC[child[0]][child[1]].name)
              return problem.ZC[child[0]][child[1]].name
          if len(frontier) < limit:
              #return problem.solution(problem.ZC[child[0]][child[1]])
            frontier.appendleft(child)
          #print('child before adding',child)
            explored.add(child)

In [ ]:
    frontier = deque()
    frontier.append(tuple([1,2]))
    while frontier:
      node = frontier.popleft()
    print(node)

(1, 2)


In [ ]:
import sys
def ids(problem):
    for depth in range(1,sys.maxsize):
        result = DLS(problem, depth)
        if result != None:
            return result

In [ ]:
init_state = 'room 207'
goal_state = 'security office'

first_try = ZC_problem(init_state,goal_state)
%prun ids(first_try)

now we are at Dorms
now we are at DO_NA
now we are at Nano
now we are at NA_ON
now we are at OneStopShop
now we are at road9
Now we reached  Gate A
 

## Greedy

In [ ]:
from heapq import heappush, heappop
from itertools import count
counter = count()
path=[]
def greedy_best_first(problem, verbose=False):
    '''Greedy best-first search implementation.'''
    problem.generate_h(problem.init_state,problem.goal_state)
    i,j = problem.init_state[0],problem.init_state[1]
    #frontier = deque()
    #frontier.append(tuple(problem.init_state))
    frontier = [(None, None, (problem.init_state))]
    explored = set()
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        #print('the smallest:  ',heappop(frontier)[2])
        #print('FF     ', frontier[0])
        _, _, node = heappop(frontier)

        print('I am at       ',problem.ZC[node[0]][node[1]].name)
        path.append(node)
        if node in explored: continue
        if problem.goal_test(node):
            #break
            #print('gf')
            return
        explored.add(node)
        print('node:     ', node[0],',',node[1])
        #(ZC_problem[node[0]][node[1]]).h
        for action in problem.actions((node)):
          #problem.actions((i,j))
          #
            child = problem.result(node, action)

            if child not in explored:
                k, l = child[0], child[1]
                new_state = problem.ZC[k][l]
                print('POSSIBLE NODE is  ', new_state.name)
                new_value = problem.heuristic(new_state)
                heappush(frontier, (new_value, next(counter), child))

In [ ]:

init_state = 'room 207'
goal_state = 'security office'

first_try = ZC_problem(init_state,goal_state)
%prun greedy_best_first(first_try)

I am at        Dorms
node:      0 , 0
POSSIBLE NODE is   DO_NA
POSSIBLE NODE is   DO_SE
I am at        DO_NA
node:      1 , 0
POSSIBLE NODE is   Nano
POSSIBLE NODE is   road2
I am at        Nano
node:      2 , 0
POSSIBLE NODE is   NA_ON
POSSIBLE NODE is   Academic
I am at        NA_ON
node:      3 , 0
POSSIBLE NODE is   OneStopShop
POSSIBLE NODE is   Complex
I am at        OneStopShop
node:      4 , 0
POSSIBLE NODE is   road9
POSSIBLE NODE is   GA_CO
I am at        road9
node:      5 , 0
POSSIBLE NODE is   Gate A
I am at        Gate A
 

In [ ]:
%prun greedy_best_first(first_try, verbose=True)

Frontier at step 1

(None, None, (0, 0))
--------------------------------------------------------------------------------
I am at        Dorms
node:      0 , 0
POSSIBLE NODE is   DO_NA
POSSIBLE NODE is   DO_SE
Frontier at step 2

(4.12, 11, (1, 0))

(5.0, 12, (0, 1))
--------------------------------------------------------------------------------
I am at        DO_NA
node:      1 , 0
POSSIBLE NODE is   Nano
POSSIBLE NODE is   road2
Frontier at step 3

(3.16, 13, (2, 0))

(5.0, 12, (0, 1))

(4.0, 14, (1, 1))
--------------------------------------------------------------------------------
I am at        Nano
node:      2 , 0
POSSIBLE NODE is   NA_ON
POSSIBLE NODE is   Academic
Frontier at step 4

(2.24, 15, (3, 0))

(3.0, 16, (2, 1))

(4.0, 14, (1, 1))

(5.0, 12, (0, 1))
--------------------------------------------------------------------------------
I am at        NA_ON
node:      3 , 0
POSSIBLE NODE is   OneStopShop
POSSIBLE NODE is   Complex
Frontier at step 5

(1.41, 17, (4, 0))

(2.

In [ ]:
path

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 0),
 (5, 1),
 (0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 0),
 (5, 1)]

In [ ]:
def find_path_steps(path):
  actions_steps = []
  for i in range(len(path)):
    if i == len(path)-1:
      break
    else:
      if path[i][0] == path[i+1][0]:
        if path[i][1] < path[i+1][1]:
          actions_steps.append('right')
        else:
          actions_steps.append('left')
      if path[i][1] == path[i+1][1]:
        if path[i][0] < path[i+1][0]:
          actions_steps.append('down')
        else:
          actions_steps.append('up')
  return actions_steps

In [ ]:
find_path_steps(path)

['down',
 'down',
 'down',
 'down',
 'down',
 'right',
 'down',
 'down',
 'down',
 'down',
 'down',
 'right']

## A*

In [ ]:
path2=[]
def A_star(problem, verbose=False):
    '''A-star search implementation.'''
    problem.generate_h(problem.init_state,problem.goal_state)
    i,j = problem.init_state[0],problem.init_state[1]

    frontier = [(None, None, problem.init_state)]
    explored = set()
    while frontier:
        #node = frontier.popleft()
        _, _, node = heappop(frontier)
        print('I am at       ',problem.ZC[node[0]][node[1]].name)
        path2.append(node)
        if node in explored: continue
        if problem.goal_test(node):
            return
        explored.add(node)
        for action in problem.actions(node):
           child = problem.result(node, action)
           problem.ZC[node[0]][node[1]].child = problem.ZC[child[0]][child[1]]
           problem.ZC[child[0]][child[1]].parent = problem.ZC[node[0]][node[1]]
           if child not in explored:
             k, l = child[0], child[1]
             new_state = problem.ZC[k][l]
             print('Next is  ', new_state.name)
             path_cost=10
             new_value = problem.heuristic(new_state)
             heappush(frontier, (path_cost+new_value, next(counter), child))

In [ ]:
init_state = 'room 207'
goal_state = 'security office'

first_try = ZC_problem(init_state,goal_state)
%prun A_star(first_try)

I am at        Dorms
Next is   DO_NA
Next is   DO_SE
I am at        DO_NA
Next is   Nano
Next is   road2
I am at        Nano
Next is   NA_ON
Next is   Academic
I am at        NA_ON
Next is   OneStopShop
Next is   Complex
I am at        OneStopShop
Next is   road9
Next is   GA_CO
I am at        road9
Next is   Gate A
I am at        Gate A
 

In [ ]:
path2

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (5, 1)]

In [ ]:
find_path_steps(path2)

['down', 'down', 'down', 'down', 'down', 'right']

## Hill Climbing

In [ ]:
def hill_climbing(problem, verbose=True):
    '''Hill climbing search implementation.'''
    problem.generate_h(problem.init_state,problem.goal_state)

    i,j = problem.init_state[0],problem.init_state[1]
    current_state = problem.ZC[i][j]
    #print(current_state)
    current_value = problem.heuristic(current_state)
    if verbose: visualizer = Visualizer(problem)
    while True:
      if verbose: visualizer.visualize([current_state.name])
      next_state, next_value = None, None
      for action in problem.actions((i,j)):
        #print(f'action: {action}')
        k,l = problem.result((i,j), action)
        #print('??',k,l)
        new_state = problem.ZC[k][l]
        new_value = problem.heuristic(new_state)
        #print(f'next value : {new_value}')
        #print(f'current value:{current_value}')
        if next_value is None or next_value > new_value:
          next_state, next_value = new_state, new_value
          new_i,new_j = k,l
          best_action = action
      if current_value <= next_value: return current_state
      #print('now next value became : ',next_value)
      current_state, current_value = next_state, next_value
      i,j = new_i,new_j
      #print("Now i'm at",i,j)


### Testing Hill climbing

In [ ]:
init_state = ('room 207')
goal_state = ('security office')

ZC_try2 = ZC_problem(init_state,goal_state)

%prun hill_climbing(ZC_try2)

Frontier at step 1

Dorms
--------------------------------------------------------------------------------
Frontier at step 2

DO_NA
--------------------------------------------------------------------------------
Frontier at step 3

Nano
--------------------------------------------------------------------------------
Frontier at step 4

NA_ON
--------------------------------------------------------------------------------
Frontier at step 5

OneStopShop
--------------------------------------------------------------------------------
Frontier at step 6

road9
--------------------------------------------------------------------------------
Frontier at step 7

Gate A
--------------------------------------------------------------------------------
 

## Simulated Annealing

In [ ]:
from random import choice, random
from math import exp
from itertools import count

def simulated_annealing(problem, schedule, verbose=False):
  problem.generate_h(problem.init_state,problem.goal_state)
  '''Simulated annealing search implementation.'''
  i,j = problem.init_state[0],problem.init_state[1]
  current_state = problem.ZC[i][j]
  current_value = problem.heuristic(current_state)
  if verbose: visualizer = Visualizer(problem)
  for t in count():
    if verbose: visualizer.visualize([current_state.name])
    T = schedule(t)  # A function that determines the "temperature" (acceptability of a bad state) as a function of the step count
    if current_value == 0 or T == 0: return current_state.name  # Return if a goal state is found or if the temperature hits 0
    next_states = [problem.result((i,j), action) for action in problem.actions((i,j))]  # Generate all possible next states
    #print('next states ',next_states)
    while True:  # Repeat the following till the current state is updated
      next_state = choice(next_states)  # Choose a random next state
      #print("next state",next_state)
      next_value = problem.heuristic(problem.ZC[next_state[0]][next_state[1]])
      #print('next Value',next_value)
      delta = current_value - next_value
      if delta > 0 or random() < exp(delta / T):  # Accept the randomly chosen state immediately if it is better than the current state or with a probability (exponentially) proportional to the temperature and how bad it is
        #print('its better')
        (i,j), current_value = next_state, next_value
        current_state = problem.ZC[next_state[0]][next_state[1]]
        print(f"Go to : {current_state.name}")
        break

### Testing simulated annealing

In [ ]:
init_state = 'room 207' # in dorms
goal_state = "security office"  # at Gate A

ZC_sim = ZC_problem(init_state,goal_state)
%prun simulated_annealing(ZC_sim,lambda t: exp(-t))


Go to : DO_SE
Go to : service
Go to : road3
Go to : Helmy
Go to : Academic
Go to : Complex
Go to : GA_CO
Go to : Gate A
 